# Google Drive Interface Setup

In [1]:
from google.colab import drive, auth
import sys

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Pull down github repo, store in colab runtime memory
!git clone https://github.com/andrew-loeber/proj_utils_207.git

Cloning into 'proj_utils_207'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 34 (delta 10), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (34/34), 8.72 KiB | 8.72 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [4]:
# Tell Python to also look in this repo when running an import
sys.path.insert(1, "/content/proj_utils_207")

In [5]:
# Authenticate google account and give back the session access token
auth.authenticate_user()
gcloud_token = !gcloud auth print-access-token
gcloud_token

['ya29.a0AfB_byBI_6MraxzTT4p28WOH1KyUfDfBwpznC0xGnx76z9NzJP2jrRXkirPXsWLiJH5wnV-EnoY038pYJqTh6aBcbH4Le9w4WcenQaIAsh7IDlwSOVOc-d78GPa1Y31Ma2LU-ehKg-ny4J5AalAq0Cv-KGUyaCgYKATQSARASFQHsvYls5Rvwjflhy1CMR0UqdosJ4A0163']

In [6]:
from gdriveinterface import GDriveInterface
import proj_ref

In [7]:
# Show email address, username, and the path of the shared project folder on
# your Google Drive instance
gdi = GDriveInterface(gcloud_token)
shared_folder_path = gdi.shared_folder_path
gdi.__dict__

{'email': 'aloeber@berkeley.edu',
 'account': 'aloeber',
 'shared_folder_path': '/content/drive/MyDrive/207-Project'}

In [8]:
# Show name keys and paths for tracked files
proj_ref.files

{'taxonomy': 'BirdCLEF/eBird_Taxonomy_v2021.csv',
 'orig_metadata': 'BirdCLEF/train_metadata_with_duration.csv',
 'sample_metadata': 'data/sample_metadata.csv',
 'species_metadata': 'data/species_metadata.csv',
 'train_metadata': 'data/train_metadata.csv',
 'test_metadata': 'data/test_metadata.csv'}

In [9]:
# Show name keys and paths for tracked directories
proj_ref.dirs

{'orig_audio': {'path': 'BirdCLEF/train_audio'},
 'train_audio': {'path': 'data/train/audio_files'},
 'test_audio': {'path': 'data/test/audio_files'},
 'train_npy_full': {'path': 'data/train/librosa_loaded'},
 'test_npy_full': {'path': 'data/test/librosa_loaded'},
 'train_npy_loud5s': {'path': 'data/train/librosa_loaded_loudest_5sec'},
 'test_npy_loud5s': {'path': 'data/test/librosa_loaded_loudest_5sec'}}

### **Usage examples**

##### Query specific file based on named key
```
command: gdi.get_file_path('taxonomy')
returns: '/content/drive/MyDrive/207-Project/BirdCLEF/eBird_Taxonomy_v2021.csv'
```

##### Query specific directory based on named key
```
command: gdi.get_dir_path('orig_audio')
returns: '/content/drive/MyDrive/207-Project/BirdCLEF/train_audio'
```

##### Query specific sample record based on named keys for directory and file
```
command: gdi.get_sample_path('train_audio', 'barswa/XC132406.ogg')
returns: '/content/drive/MyDrive/207-Project/data/train/audio_files/barswa/XC132406.ogg'
```

##### Provide full path given a custom partial path
```
command: gdi.join_to_shared('images/sandpiper.jpeg')
returns: '/content/drive/MyDrive/207-Project/images/sandpiper.jpeg'
```


# Imports & Setup

In [10]:
import numpy as np
import pandas as pd
import pickle as pkl
import librosa
from IPython.display import Audio
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# Load Train_Val DataFrame & Encode Features

In [11]:
df = pd.read_csv(gdi.join_to_shared('3_species/train_val.csv'))

df.head()

,primary_label,filename,type,filename_npy,rating,duration_secs_32000,country,continent,data
0,eaywag1,eaywag1/XC718442.ogg,blank,eaywag1/XC718442.npy,good,12.538781,FR,EUROPE,train
1,eaywag1,eaywag1/XC675682.ogg,call,eaywag1/XC675682.npy,good,35.657000,RU,EUROPE,train
2,eaywag1,eaywag1/XC722533.ogg,blank,eaywag1/XC722533.npy,good,58.104000,RU,EUROPE,train
3,eaywag1,eaywag1/XC673617.ogg,call,eaywag1/XC673617.npy,poor,18.756000,GB,EUROPE,train
4,eaywag1,eaywag1/XC675935.ogg,call,eaywag1/XC675935.npy,good,16.666000,RU,EUROPE,train


In [14]:
df = df[df['duration_secs_32000'] >= 8]
df

,primary_label,filename,type,filename_npy,rating,duration_secs_32000,country,continent,data
0,eaywag1,eaywag1/XC718442.ogg,blank,eaywag1/XC718442.npy,good,12.538781,FR,EUROPE,train
1,eaywag1,eaywag1/XC675682.ogg,call,eaywag1/XC675682.npy,good,35.657000,RU,EUROPE,train
2,eaywag1,eaywag1/XC722533.ogg,blank,eaywag1/XC722533.npy,good,58.104000,RU,EUROPE,train
3,eaywag1,eaywag1/XC673617.ogg,call,eaywag1/XC673617.npy,poor,18.756000,GB,EUROPE,train
4,eaywag1,eaywag1/XC675935.ogg,call,eaywag1/XC675935.npy,good,16.666000,RU,EUROPE,train
...,...,...,...,...,...,...,...,...,...
934,barswa,barswa/XC194688.ogg,call,barswa/XC194688.npy,good,46.445719,KR,ASIA,val
935,barswa,barswa/XC571315.ogg,call,barswa/XC571315.npy,good,52.636750,GB,EUROPE,val
936,barswa,barswa/XC192547.ogg,call,barswa/XC192547.npy,good,22.800000,UNKNOWN,UNKNOWN,val
937,barswa,barswa/XC394763.ogg,call,barswa/XC394763.npy,good,37.642437,NL,EUROPE,val


In [15]:
df.reset_index(drop=True, inplace=True)
df

,primary_label,filename,type,filename_npy,rating,duration_secs_32000,country,continent,data
0,eaywag1,eaywag1/XC718442.ogg,blank,eaywag1/XC718442.npy,good,12.538781,FR,EUROPE,train
1,eaywag1,eaywag1/XC675682.ogg,call,eaywag1/XC675682.npy,good,35.657000,RU,EUROPE,train
2,eaywag1,eaywag1/XC722533.ogg,blank,eaywag1/XC722533.npy,good,58.104000,RU,EUROPE,train
3,eaywag1,eaywag1/XC673617.ogg,call,eaywag1/XC673617.npy,poor,18.756000,GB,EUROPE,train
4,eaywag1,eaywag1/XC675935.ogg,call,eaywag1/XC675935.npy,good,16.666000,RU,EUROPE,train
...,...,...,...,...,...,...,...,...,...
789,barswa,barswa/XC194688.ogg,call,barswa/XC194688.npy,good,46.445719,KR,ASIA,val
790,barswa,barswa/XC571315.ogg,call,barswa/XC571315.npy,good,52.636750,GB,EUROPE,val
791,barswa,barswa/XC192547.ogg,call,barswa/XC192547.npy,good,22.800000,UNKNOWN,UNKNOWN,val
792,barswa,barswa/XC394763.ogg,call,barswa/XC394763.npy,good,37.642437,NL,EUROPE,val


In [16]:
cols_to_oe = ['type', 'country', 'continent']
col_to_le = 'primary_label'

In [17]:
oe = OrdinalEncoder(dtype=np.uint16)
le = LabelEncoder()

In [18]:
oe.fit(df[cols_to_oe])

OrdinalEncoder(dtype=<class 'numpy.uint16'>)

In [19]:
le.fit(df[col_to_le])

LabelEncoder()

In [20]:
oe.categories_

[array(['blank', 'both', 'call', 'song'], dtype=object),
 array(['AE', 'AR', 'AT', 'BA', 'BE', 'BG', 'BO', 'BR', 'CA', 'CD', 'CH',
        'CN', 'CY', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GE', 'GR',
        'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IR', 'IT', 'JP', 'KR', 'KZ',
        'LA', 'LV', 'MA', 'MN', 'MX', 'MY', 'NL', 'NO', 'OM', 'PL', 'PT',
        'RU', 'SA', 'SE', 'SG', 'TH', 'TR', 'TW', 'UA', 'UNKNOWN', 'US',
        'UZ', 'ZA', 'ZM'], dtype=object),
 array(['AFRICA', 'AMERICAS', 'ASIA', 'EUROPE', 'UNKNOWN'], dtype=object)]

In [21]:
df[[col + '_enc' for col in cols_to_oe]] = oe.transform(df[cols_to_oe])
df[col_to_le + '_enc'] = le.transform(df[col_to_le])
df

,primary_label,filename,type,filename_npy,rating,duration_secs_32000,country,continent,data,type_enc,country_enc,continent_enc,primary_label_enc
0,eaywag1,eaywag1/XC718442.ogg,blank,eaywag1/XC718442.npy,good,12.538781,FR,EUROPE,train,0,18,3,2
1,eaywag1,eaywag1/XC675682.ogg,call,eaywag1/XC675682.npy,good,35.657000,RU,EUROPE,train,2,44,3,2
2,eaywag1,eaywag1/XC722533.ogg,blank,eaywag1/XC722533.npy,good,58.104000,RU,EUROPE,train,0,44,3,2
3,eaywag1,eaywag1/XC673617.ogg,call,eaywag1/XC673617.npy,poor,18.756000,GB,EUROPE,train,2,19,3,2
4,eaywag1,eaywag1/XC675935.ogg,call,eaywag1/XC675935.npy,good,16.666000,RU,EUROPE,train,2,44,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,barswa,barswa/XC194688.ogg,call,barswa/XC194688.npy,good,46.445719,KR,ASIA,val,2,31,2,0
790,barswa,barswa/XC571315.ogg,call,barswa/XC571315.npy,good,52.636750,GB,EUROPE,val,2,19,3,0
791,barswa,barswa/XC192547.ogg,call,barswa/XC192547.npy,good,22.800000,UNKNOWN,UNKNOWN,val,2,52,4,0
792,barswa,barswa/XC394763.ogg,call,barswa/XC394763.npy,good,37.642437,NL,EUROPE,val,2,39,3,0


In [22]:
df = df[[
  'data'
  , 'primary_label'
  , 'primary_label_enc'
  , 'filename'
  , 'filename_npy'
  , 'rating'
  , 'type'
  , 'type_enc'
  , 'country'
  , 'country_enc'
  , 'continent'
  , 'continent_enc'
  , 'duration_secs_32000'
]]

df

,data,primary_label,primary_label_enc,filename,filename_npy,rating,type,type_enc,country,country_enc,continent,continent_enc,duration_secs_32000
0,train,eaywag1,2,eaywag1/XC718442.ogg,eaywag1/XC718442.npy,good,blank,0,FR,18,EUROPE,3,12.538781
1,train,eaywag1,2,eaywag1/XC675682.ogg,eaywag1/XC675682.npy,good,call,2,RU,44,EUROPE,3,35.657000
2,train,eaywag1,2,eaywag1/XC722533.ogg,eaywag1/XC722533.npy,good,blank,0,RU,44,EUROPE,3,58.104000
3,train,eaywag1,2,eaywag1/XC673617.ogg,eaywag1/XC673617.npy,poor,call,2,GB,19,EUROPE,3,18.756000
4,train,eaywag1,2,eaywag1/XC675935.ogg,eaywag1/XC675935.npy,good,call,2,RU,44,EUROPE,3,16.666000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,val,barswa,0,barswa/XC194688.ogg,barswa/XC194688.npy,good,call,2,KR,31,ASIA,2,46.445719
790,val,barswa,0,barswa/XC571315.ogg,barswa/XC571315.npy,good,call,2,GB,19,EUROPE,3,52.636750
791,val,barswa,0,barswa/XC192547.ogg,barswa/XC192547.npy,good,call,2,UNKNOWN,52,UNKNOWN,4,22.800000
792,val,barswa,0,barswa/XC394763.ogg,barswa/XC394763.npy,good,call,2,NL,39,EUROPE,3,37.642437


In [23]:
df.rename({'duration_secs_32000': 'file_duration_s'}, axis=1, inplace=True)
df

,data,primary_label,primary_label_enc,filename,filename_npy,rating,type,type_enc,country,country_enc,continent,continent_enc,file_duration_s
0,train,eaywag1,2,eaywag1/XC718442.ogg,eaywag1/XC718442.npy,good,blank,0,FR,18,EUROPE,3,12.538781
1,train,eaywag1,2,eaywag1/XC675682.ogg,eaywag1/XC675682.npy,good,call,2,RU,44,EUROPE,3,35.657000
2,train,eaywag1,2,eaywag1/XC722533.ogg,eaywag1/XC722533.npy,good,blank,0,RU,44,EUROPE,3,58.104000
3,train,eaywag1,2,eaywag1/XC673617.ogg,eaywag1/XC673617.npy,poor,call,2,GB,19,EUROPE,3,18.756000
4,train,eaywag1,2,eaywag1/XC675935.ogg,eaywag1/XC675935.npy,good,call,2,RU,44,EUROPE,3,16.666000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,val,barswa,0,barswa/XC194688.ogg,barswa/XC194688.npy,good,call,2,KR,31,ASIA,2,46.445719
790,val,barswa,0,barswa/XC571315.ogg,barswa/XC571315.npy,good,call,2,GB,19,EUROPE,3,52.636750
791,val,barswa,0,barswa/XC192547.ogg,barswa/XC192547.npy,good,call,2,UNKNOWN,52,UNKNOWN,4,22.800000
792,val,barswa,0,barswa/XC394763.ogg,barswa/XC394763.npy,good,call,2,NL,39,EUROPE,3,37.642437


# Prepare Train & Val DataFrames

In [24]:
train_df = df[df['data'] == 'train'].copy()
val_df = df[df['data'] == 'val'].copy()

In [25]:
train_df.head()

,data,primary_label,primary_label_enc,filename,filename_npy,rating,type,type_enc,country,country_enc,continent,continent_enc,file_duration_s
0,train,eaywag1,2,eaywag1/XC718442.ogg,eaywag1/XC718442.npy,good,blank,0,FR,18,EUROPE,3,12.538781
1,train,eaywag1,2,eaywag1/XC675682.ogg,eaywag1/XC675682.npy,good,call,2,RU,44,EUROPE,3,35.657000
2,train,eaywag1,2,eaywag1/XC722533.ogg,eaywag1/XC722533.npy,good,blank,0,RU,44,EUROPE,3,58.104000
3,train,eaywag1,2,eaywag1/XC673617.ogg,eaywag1/XC673617.npy,poor,call,2,GB,19,EUROPE,3,18.756000
4,train,eaywag1,2,eaywag1/XC675935.ogg,eaywag1/XC675935.npy,good,call,2,RU,44,EUROPE,3,16.666000


In [26]:
train_df = train_df.sample(frac=1, random_state=1234)
train_df.reset_index(drop=True, inplace=True)
train_df.head()

,data,primary_label,primary_label_enc,filename,filename_npy,rating,type,type_enc,country,country_enc,continent,continent_enc,file_duration_s
0,train,barswa,0,barswa/XC659151.ogg,barswa/XC659151.npy,good,call,2,PT,43,EUROPE,3,47.438375
1,train,eaywag1,2,eaywag1/XC738300.ogg,eaywag1/XC738300.npy,good,blank,0,FR,18,EUROPE,3,18.408000
2,train,eaywag1,2,eaywag1/XC675971.ogg,eaywag1/XC675971.npy,good,song,3,RU,44,EUROPE,3,18.364000
3,train,comsan,1,comsan/XC606397.ogg,comsan/XC606397.npy,good,call,2,FR,18,EUROPE,3,52.584000
4,train,comsan,1,comsan/XC469618.ogg,comsan/XC469618.npy,poor,call,2,PL,42,EUROPE,3,37.276750


In [27]:
val_df.head()

,data,primary_label,primary_label_enc,filename,filename_npy,rating,type,type_enc,country,country_enc,continent,continent_enc,file_duration_s
560,val,eaywag1,2,eaywag1/XC182483.ogg,eaywag1/XC182483.npy,good,call,2,KZ,32,ASIA,2,21.072
561,val,eaywag1,2,eaywag1/XC675723.ogg,eaywag1/XC675723.npy,good,call,2,RU,44,EUROPE,3,83.330
562,val,eaywag1,2,eaywag1/XC589278.ogg,eaywag1/XC589278.npy,good,call,2,FR,18,EUROPE,3,13.320
563,val,eaywag1,2,eaywag1/XC634278.ogg,eaywag1/XC634278.npy,good,call,2,PT,43,EUROPE,3,26.616
564,val,eaywag1,2,eaywag1/XC584625.ogg,eaywag1/XC584625.npy,good,call,2,EE,15,EUROPE,3,116.375


In [28]:
val_df = val_df.sample(frac=1, random_state=1234)
val_df.reset_index(drop=True, inplace=True)
val_df.head()

,data,primary_label,primary_label_enc,filename,filename_npy,rating,type,type_enc,country,country_enc,continent,continent_enc,file_duration_s
0,val,comsan,1,comsan/XC157141.ogg,comsan/XC157141.npy,good,call,2,NO,40,EUROPE,3,46.920000
1,val,barswa,0,barswa/XC480227.ogg,barswa/XC480227.npy,good,call,2,US,53,AMERICAS,1,13.824000
2,val,eaywag1,2,eaywag1/XC417163.ogg,eaywag1/XC417163.npy,good,song,3,UA,51,EUROPE,3,91.896000
3,val,comsan,1,comsan/XC638592.ogg,comsan/XC638592.npy,good,call,2,GB,19,EUROPE,3,12.624000
4,val,eaywag1,2,eaywag1/XC571987.ogg,eaywag1/XC571987.npy,good,call,2,BE,4,EUROPE,3,57.573875


# Load & save train audio to file

In [29]:
def load_audio(df
               , print_iters=20
               , dir_key='train_npy_full'):

  recs = len(df)
  print(f'{recs} files to load')
  print()

  audio_files = []

  for i in range(recs):
    if (i+1) % print_iters == 0:
      print(f'{i+1} of {recs}')
    row = dict(df.iloc[i])
    path = gdi.get_sample_path(dir_key=dir_key, file_key=row['filename_npy'])
    audio = np.load(path)
    audio_files.append(audio)

  print(f'{recs} of {recs}...DONE')

  return audio_files


In [30]:
train_files = load_audio(train_df)

560 files to load

20 of 560
40 of 560
60 of 560
80 of 560
100 of 560
120 of 560
140 of 560
160 of 560
180 of 560
200 of 560
220 of 560
240 of 560
260 of 560
280 of 560
300 of 560
320 of 560
340 of 560
360 of 560
380 of 560
400 of 560
420 of 560
440 of 560
460 of 560
480 of 560
500 of 560
520 of 560
540 of 560
560 of 560
560 of 560...DONE


In [31]:
train_files[0:5]

[array([-2.4908164e-05, -1.4384452e-05, -1.4580583e-06, ...,
        -8.7092584e-04, -3.0176807e-04,  3.4509483e-04], dtype=float32),
 array([ 7.4012205e-06,  1.1723838e-05, -1.5496393e-05, ...,
        -5.9779547e-02, -5.4983735e-02, -5.6934610e-02], dtype=float32),
 array([ 1.6528298e-05,  1.5403963e-05, -3.3902211e-08, ...,
         6.2227668e-04,  2.8527057e-04, -3.5850500e-04], dtype=float32),
 array([-4.4190092e-06, -1.9313302e-06,  1.6307749e-05, ...,
         2.1933345e-05, -6.9923117e-06, -2.0051375e-06], dtype=float32),
 array([-1.4243425e-05,  4.3990362e-06,  1.1153301e-05, ...,
         8.4286712e-06, -9.7262018e-06, -5.3474714e-06], dtype=float32)]

In [32]:
sample_num = 2
sample_info = train_df.iloc[sample_num]
sample_info

data                                train
primary_label                     eaywag1
primary_label_enc                       2
filename             eaywag1/XC675971.ogg
filename_npy         eaywag1/XC675971.npy
rating                               good
type                                 song
type_enc                                3
country                                RU
country_enc                            44
continent                          EUROPE
continent_enc                           3
file_duration_s                    18.364
Name: 2, dtype: object

In [33]:
Audio(train_files[sample_num], rate=16000)

In [34]:
sample_filepath = gdi.get_sample_path('train_npy_full', sample_info['filename_npy'])
sample_npy = np.load(sample_filepath)
Audio(sample_npy, rate=16000)

In [38]:
train_files_filepath = gdi.join_to_shared('3_species/8_sec/train/files.pkl')
with open(train_files_filepath, 'wb') as file:
  pkl.dump(train_files, file)

In [39]:
with open(train_files_filepath, 'rb') as file:
  train_files_loaded = pkl.load(file)

In [40]:
train_files_loaded[0:5]

[array([-2.4908164e-05, -1.4384452e-05, -1.4580583e-06, ...,
        -8.7092584e-04, -3.0176807e-04,  3.4509483e-04], dtype=float32),
 array([ 7.4012205e-06,  1.1723838e-05, -1.5496393e-05, ...,
        -5.9779547e-02, -5.4983735e-02, -5.6934610e-02], dtype=float32),
 array([ 1.6528298e-05,  1.5403963e-05, -3.3902211e-08, ...,
         6.2227668e-04,  2.8527057e-04, -3.5850500e-04], dtype=float32),
 array([-4.4190092e-06, -1.9313302e-06,  1.6307749e-05, ...,
         2.1933345e-05, -6.9923117e-06, -2.0051375e-06], dtype=float32),
 array([-1.4243425e-05,  4.3990362e-06,  1.1153301e-05, ...,
         8.4286712e-06, -9.7262018e-06, -5.3474714e-06], dtype=float32)]

In [41]:
Audio(train_files_loaded[sample_num], rate=16000)

In [42]:
del train_files
del train_files_loaded

# Load & frame audio

In [43]:
def load_audio_frames(
    df
    , print_iters=20
    , dir_key='train_npy_full'
    , frame_length_s=8.0
    , hop_length_s=4.0
    , sr=16000
    ):

  recs = len(df)
  frame_length = int(frame_length_s * sr)
  hop_length = int(hop_length_s * sr)

  print(f'{recs} files')
  print(f'{frame_length_s}s frames with {hop_length_s}s overlap')
  print()

  array_sizes = []
  all_frames = []
  frames_info = []

  for i in range(recs):
    if (i+1) % print_iters == 0:
      print(f'{i+1} of {recs}')
    row = dict(df.iloc[i])
    path = gdi.get_sample_path(dir_key=dir_key, file_key=row['filename_npy'])
    audio_full = np.load(path)
    array_sizes.append(len(audio_full))
    audio_frames = np.array(
        tf.signal.frame(audio_full, frame_length, hop_length, pad_end=False)
    )
    all_frames.extend(audio_frames)
    num_frames = len(audio_frames)
    frame_num = 1
    for frame in audio_frames:
      frame_dict = row.copy()
      frame_dict.update(
          {
              'frame_length_s':   frame_length_s
              , 'hop_length_s':   hop_length_s
              , 'total_frames':   num_frames
              , 'frame_id':       frame_num
          }
      )
      frames_info.append(frame_dict)
      frame_num += 1

  print(f'{recs} of {recs}...DONE')
  print()
  print(f'{len(all_frames)} total frames created')

  return array_sizes, np.array(all_frames), pd.DataFrame(frames_info)


In [44]:
train_sizes, train_frames, train_frames_df = load_audio_frames(train_df)

560 files
8.0s frames with 4.0s overlap

20 of 560
40 of 560
60 of 560
80 of 560
100 of 560
120 of 560
140 of 560
160 of 560
180 of 560
200 of 560
220 of 560
240 of 560
260 of 560
280 of 560
300 of 560
320 of 560
340 of 560
360 of 560
380 of 560
400 of 560
420 of 560
440 of 560
460 of 560
480 of 560
500 of 560
520 of 560
540 of 560
560 of 560
560 of 560...DONE

4555 total frames created


In [45]:
train_sizes[0:10]

[759014,
 294528,
 293824,
 841344,
 596428,
 173035,
 680832,
 205636,
 385920,
 417959]

In [46]:
train_frames.shape

(4555, 128000)

In [47]:
train_frames_df

,data,primary_label,primary_label_enc,filename,filename_npy,rating,type,type_enc,country,country_enc,continent,continent_enc,file_duration_s,frame_length_s,hop_length_s,total_frames,frame_id
0,train,barswa,0,barswa/XC659151.ogg,barswa/XC659151.npy,good,call,2,PT,43,EUROPE,3,47.438375,8.0,4.0,10,1
1,train,barswa,0,barswa/XC659151.ogg,barswa/XC659151.npy,good,call,2,PT,43,EUROPE,3,47.438375,8.0,4.0,10,2
2,train,barswa,0,barswa/XC659151.ogg,barswa/XC659151.npy,good,call,2,PT,43,EUROPE,3,47.438375,8.0,4.0,10,3
3,train,barswa,0,barswa/XC659151.ogg,barswa/XC659151.npy,good,call,2,PT,43,EUROPE,3,47.438375,8.0,4.0,10,4
4,train,barswa,0,barswa/XC659151.ogg,barswa/XC659151.npy,good,call,2,PT,43,EUROPE,3,47.438375,8.0,4.0,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4550,train,comsan,1,comsan/XC554068.ogg,comsan/XC554068.npy,poor,call,2,FI,17,EUROPE,3,25.600000,8.0,4.0,5,1
4551,train,comsan,1,comsan/XC554068.ogg,comsan/XC554068.npy,poor,call,2,FI,17,EUROPE,3,25.600000,8.0,4.0,5,2
4552,train,comsan,1,comsan/XC554068.ogg,comsan/XC554068.npy,poor,call,2,FI,17,EUROPE,3,25.600000,8.0,4.0,5,3
4553,train,comsan,1,comsan/XC554068.ogg,comsan/XC554068.npy,poor,call,2,FI,17,EUROPE,3,25.600000,8.0,4.0,5,4


In [48]:
train_df['file_duration_samples'] = train_sizes
train_df

,data,primary_label,primary_label_enc,filename,filename_npy,rating,type,type_enc,country,country_enc,continent,continent_enc,file_duration_s,file_duration_samples
0,train,barswa,0,barswa/XC659151.ogg,barswa/XC659151.npy,good,call,2,PT,43,EUROPE,3,47.438375,759014
1,train,eaywag1,2,eaywag1/XC738300.ogg,eaywag1/XC738300.npy,good,blank,0,FR,18,EUROPE,3,18.408000,294528
2,train,eaywag1,2,eaywag1/XC675971.ogg,eaywag1/XC675971.npy,good,song,3,RU,44,EUROPE,3,18.364000,293824
3,train,comsan,1,comsan/XC606397.ogg,comsan/XC606397.npy,good,call,2,FR,18,EUROPE,3,52.584000,841344
4,train,comsan,1,comsan/XC469618.ogg,comsan/XC469618.npy,poor,call,2,PL,42,EUROPE,3,37.276750,596428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,train,comsan,1,comsan/XC738502.ogg,comsan/XC738502.npy,good,blank,0,FR,18,EUROPE,3,47.904000,766464
556,train,comsan,1,comsan/XC417694.ogg,comsan/XC417694.npy,good,call,2,DE,13,EUROPE,3,12.456000,199296
557,train,comsan,1,comsan/XC501615.ogg,comsan/XC501615.npy,good,call,2,GB,19,EUROPE,3,11.389375,182230
558,train,eaywag1,2,eaywag1/XC507677.ogg,eaywag1/XC507677.npy,poor,call,2,GB,19,EUROPE,3,17.397562,278361


In [49]:
val_sizes, val_frames, val_frames_df = load_audio_frames(val_df)

234 files
8.0s frames with 4.0s overlap

20 of 234
40 of 234
60 of 234
80 of 234
100 of 234
120 of 234
140 of 234
160 of 234
180 of 234
200 of 234
220 of 234
234 of 234...DONE

1785 total frames created


In [50]:
val_sizes[0:10]

[750720,
 221184,
 1470336,
 201984,
 921182,
 182016,
 883584,
 1105084,
 273760,
 229872]

In [51]:
val_frames.shape

(1785, 128000)

In [52]:
val_frames_df

,data,primary_label,primary_label_enc,filename,filename_npy,rating,type,type_enc,country,country_enc,continent,continent_enc,file_duration_s,frame_length_s,hop_length_s,total_frames,frame_id
0,val,comsan,1,comsan/XC157141.ogg,comsan/XC157141.npy,good,call,2,NO,40,EUROPE,3,46.920000,8.0,4.0,10,1
1,val,comsan,1,comsan/XC157141.ogg,comsan/XC157141.npy,good,call,2,NO,40,EUROPE,3,46.920000,8.0,4.0,10,2
2,val,comsan,1,comsan/XC157141.ogg,comsan/XC157141.npy,good,call,2,NO,40,EUROPE,3,46.920000,8.0,4.0,10,3
3,val,comsan,1,comsan/XC157141.ogg,comsan/XC157141.npy,good,call,2,NO,40,EUROPE,3,46.920000,8.0,4.0,10,4
4,val,comsan,1,comsan/XC157141.ogg,comsan/XC157141.npy,good,call,2,NO,40,EUROPE,3,46.920000,8.0,4.0,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,val,eaywag1,2,eaywag1/XC286395.ogg,eaywag1/XC286395.npy,good,call,2,ES,16,EUROPE,3,10.579594,8.0,4.0,1,1
1781,val,barswa,0,barswa/XC270309.ogg,barswa/XC270309.npy,good,call,2,RU,44,EUROPE,3,17.554281,8.0,4.0,3,1
1782,val,barswa,0,barswa/XC270309.ogg,barswa/XC270309.npy,good,call,2,RU,44,EUROPE,3,17.554281,8.0,4.0,3,2
1783,val,barswa,0,barswa/XC270309.ogg,barswa/XC270309.npy,good,call,2,RU,44,EUROPE,3,17.554281,8.0,4.0,3,3


In [53]:
val_df['file_duration_samples'] = val_sizes
val_df

,data,primary_label,primary_label_enc,filename,filename_npy,rating,type,type_enc,country,country_enc,continent,continent_enc,file_duration_s,file_duration_samples
0,val,comsan,1,comsan/XC157141.ogg,comsan/XC157141.npy,good,call,2,NO,40,EUROPE,3,46.920000,750720
1,val,barswa,0,barswa/XC480227.ogg,barswa/XC480227.npy,good,call,2,US,53,AMERICAS,1,13.824000,221184
2,val,eaywag1,2,eaywag1/XC417163.ogg,eaywag1/XC417163.npy,good,song,3,UA,51,EUROPE,3,91.896000,1470336
3,val,comsan,1,comsan/XC638592.ogg,comsan/XC638592.npy,good,call,2,GB,19,EUROPE,3,12.624000,201984
4,val,eaywag1,2,eaywag1/XC571987.ogg,eaywag1/XC571987.npy,good,call,2,BE,4,EUROPE,3,57.573875,921182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,val,barswa,0,barswa/XC631959.ogg,barswa/XC631959.npy,good,both,1,MX,37,AMERICAS,1,28.943687,463099
230,val,eaywag1,2,eaywag1/XC714228.ogg,eaywag1/XC714228.npy,good,blank,0,FR,18,EUROPE,3,56.664000,906624
231,val,eaywag1,2,eaywag1/XC286395.ogg,eaywag1/XC286395.npy,good,call,2,ES,16,EUROPE,3,10.579594,169274
232,val,barswa,0,barswa/XC270309.ogg,barswa/XC270309.npy,good,call,2,RU,44,EUROPE,3,17.554281,280869


# Save train dfs, frames, and labels to drive as npy files

In [54]:
train_df.to_csv(
    path_or_buf=gdi.join_to_shared('3_species/8_sec/train/file_df.csv')
    ,index=False
)

In [55]:
train_labels = np.array(train_df['primary_label_enc'])
train_labels

array([0, 2, 2, 1, 1, 1, 2, 1, 2, 0, 1, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 2, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0,
       0, 0, 2, 1, 0, 1, 1, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 1, 1, 1, 2,
       2, 2, 0, 2, 0, 2, 1, 1, 2, 0, 1, 0, 1, 0, 0, 0, 2, 2, 1, 2, 0, 0,
       1, 1, 0, 2, 1, 1, 1, 1, 2, 0, 2, 0, 1, 0, 1, 1, 2, 1, 1, 2, 1, 0,
       1, 2, 1, 0, 1, 2, 0, 0, 0, 0, 2, 2, 2, 1, 2, 1, 2, 2, 0, 1, 0, 2,
       2, 1, 2, 0, 2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 2, 0, 0, 0,
       1, 1, 0, 2, 1, 0, 1, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 2, 2, 0, 0, 1, 1, 1, 0, 2, 0, 1, 0, 1, 2, 0, 2, 1, 2, 1, 2,
       2, 2, 1, 2, 2, 2, 2, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0, 2, 1, 0, 1, 1,
       1, 2, 2, 1, 1, 2, 1, 0, 1, 0, 0, 0, 1, 0, 2, 2, 1, 1, 0, 0, 2, 1,
       1, 2, 2, 0, 1, 1, 1, 2, 0, 2, 2, 2, 0, 1, 0, 0, 1, 1, 1, 1, 2, 0,
       0, 0, 1, 0, 2, 1, 1, 1, 2, 2, 2, 0, 0, 1, 2, 2, 2, 2, 0, 0, 2, 0,
       0, 2, 2, 1, 2, 1, 2, 1, 0, 0, 1, 2, 2, 1, 0,

In [56]:
np.save(
    file=gdi.join_to_shared('3_species/8_sec/train/file_labels')
    , arr=train_labels
)

In [57]:
train_frames_df.to_csv(
    path_or_buf=gdi.join_to_shared('3_species/8_sec/train/frame_df.csv')
    ,index=False
)

In [58]:
train_frame_labels = np.array(train_frames_df['primary_label_enc'])
train_frame_labels

array([0, 0, 0, ..., 1, 1, 1])

In [59]:
np.save(
    file=gdi.join_to_shared('3_species/8_sec/train/frame_labels')
    , arr=train_frame_labels
)

In [60]:
np.save(
    file=gdi.join_to_shared('3_species/8_sec/train/frames')
    , arr=train_frames
)

# Save validation dfs, frames, and labels to drive as npy files

In [61]:
val_df.to_csv(
    path_or_buf=gdi.join_to_shared('3_species/8_sec/val/file_df.csv')
    ,index=False
)

In [62]:
val_labels = np.array(val_df['primary_label_enc'])
val_labels

array([1, 0, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 2, 2, 0, 2, 2, 2, 1, 2, 1, 2,
       1, 0, 1, 1, 1, 0, 0, 0, 2, 1, 0, 0, 1, 2, 1, 1, 2, 0, 1, 1, 0, 2,
       2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 1, 2, 0, 2, 0, 1, 2, 0, 1, 0, 0, 0,
       1, 2, 1, 1, 2, 0, 1, 2, 0, 2, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 1, 2,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 2, 2, 1, 2, 0, 2, 1,
       2, 2, 2, 0, 0, 0, 0, 0, 2, 1, 1, 2, 0, 2, 2, 1, 2, 2, 2, 1, 2, 0,
       0, 0, 0, 0, 2, 0, 1, 0, 1, 2, 1, 1, 1, 1, 1, 2, 1, 0, 2, 2, 2, 0,
       1, 1, 2, 2, 2, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 2, 0, 1, 2, 2, 2, 2,
       2, 1, 1, 0, 1, 0, 1, 0, 2, 1, 1, 1, 0, 0, 2, 2, 1, 1, 0, 1, 2, 2,
       2, 0, 2, 1, 1, 0, 0, 2, 2, 2, 2, 0, 1, 0, 1, 2, 1, 1, 0, 1, 2, 1,
       0, 0, 2, 1, 2, 0, 1, 1, 1, 0, 2, 2, 0, 2])

In [63]:
np.save(
    file=gdi.join_to_shared('3_species/8_sec/val/file_labels')
    , arr=val_labels
)

In [64]:
val_frames_df.to_csv(
    path_or_buf=gdi.join_to_shared('3_species/8_sec/val/frame_df.csv')
    ,index=False
)

In [65]:
val_frame_labels = np.array(val_frames_df['primary_label_enc'])
val_frame_labels

array([1, 1, 1, ..., 0, 0, 2])

In [66]:
np.save(
    file=gdi.join_to_shared('3_species/8_sec/val/frame_labels')
    , arr=val_frame_labels
)

In [67]:
np.save(
    file=gdi.join_to_shared('3_species/8_sec/val/frames')
    , arr=val_frames
)